# Scale

In [1]:
from adaptivetuning import Scale

<IPython.core.display.Javascript object>

A scale object manages a mapping from pitches to frequencies.

In [4]:
s = Scale()

Pitches can be given in midi numbers.

In [5]:
s[69]

440.0

(The default scale is 12TET)

Or as strings:

In [7]:
s['A4']

440.0

In [8]:
s['A#4']

466.1637615180899

In [9]:
s['Bb4']

466.1637615180899

You can change a scale one pitch at a time.

In [10]:
s['A4'] = 123
s[69]

123

Or all pitches of the same class at once.

In [13]:
s.tune_pitchclass('A4', 400)
s['A3']

200.0

In [14]:
s['A5']

800

This depends on the size of the octave and the number of pitches per octave. The defaults are 2 and 12, but we can change that:

In [16]:
s.octave_interval = 3
s.pitches_per_octave = 10
s.tune_pitchclass(69, 300)
s[59]

100.0

In [17]:
s[79]

900

Instead of calling tune_pitchclass for every pitch of the scale we can use tune_all.
We have to give tune_all a list with a frequency for every pitch in our scale beginning with the reference pitch.

In [24]:
s.reference_pitch  # The default reference pitch is A4

69

In [21]:
s.tune_all([1,2,3,4,5,6,7,8,9,10])  # remember we have changed the pitches per octave to 10!
s[79]

3

In [23]:
s[80]

6

Usually we define a scale by giving a frequency to the reference pitch and define the other pitches by giving the interval they form with the reference pitch.

In [25]:
s.reference_pitch = 'C4'
s.reference_frequency = 260
s.pitches_per_octave = 12
s.octave_interval = 2

Important: changing the reference does not change the scale!

In [26]:
s['C4']

0.6666666666666666

We have to tune the pitch C4, for example by specifying its interval with the reference pitch, just like any other pitch.

In [29]:
s.tune_pitch_by_interval('C4', 1)

In [30]:
s['C4']

260

Again, we can tune whole pitchesclasses or all pitches of the octave at once.

In [31]:
s.tune_all_by_interval([1, 15/16, 9/8, 6/5, 5/4, 4/3, 45/32, 3/2, 8/5, 5/3, 9/5, 15/8])

In [36]:
s[60 : 60 + 13]  # A just scale with tonic C

[260,
 243.75,
 292.5,
 312.0,
 325.0,
 346.66666666666663,
 365.625,
 390.0,
 416.0,
 433.33333333333337,
 468.0,
 487.5,
 520]

We can specify intervals in cents instead of as frequency ratios. A number of historic tunings in cents are stored in the Scale class.

In [37]:
Scale.tunings_in_cents['Werkmeister 4']

[0, 82, 196, 294, 392, 498, 588, 694, 784, 890, 1004, 1086]

In [39]:
s.tune_all_by_interval_in_cents(Scale.tunings_in_cents['Werkmeister 4'])

In [40]:
Scale.freq_ratio_to_cents(s['E5'] / s['C5'])

391.9999999999999

If we want to specify only a subset of the midi-keys we can. Let's look at a more complicated example:

We want to specify only the diatonic pitches in C major, such that A4 = 440 Hz

In [46]:
diatonic_pitches = ['C4','D4', 'E4', 'F4', 'G4', 'A4', 'B4']
diatonic_intervals = [1, 9/8, 5/4, 4/3, 3/2, 5/3, 15/8]
s = Scale(reference_pitch=diatonic_pitches[0], reference_frequency=440/diatonic_intervals[5],
          pitches_per_octave=len(diatonic_pitches), octave_interval=2,
          specified_pitches=diatonic_pitches, init_ET=False)

The scale has only 7 pitches and all are set to 0.

In [47]:
s.specified_pitches

dict_keys([60, 62, 64, 65, 67, 69, 71])

In [48]:
s['C4']

0

Let's tune them all to Ptolemy's syntotic diatonic scale.

In [50]:
s.tune_all_by_interval(diatonic_intervals, diatonic_pitches)
s['C4']

264.0

In [51]:
s['A4']

440.0

If the scale is asked for a pitch that is not specified, it returns the next specified pitch - this way we can use the usual keyboard layout to play our scale, without getting errors when we accidentally hit a black key.

In [53]:
s['C#4']

264.0

In [54]:
s['C3']

264.0

Now we can generalize our octave of 7 specified pitches to the whole midi range.

In [56]:
s.generalize_to_midi_range(diatonic_pitches)

In [57]:
s['C3']

132.0

Another example: The Bohlen-Pierce scale - A stretched scale (octave = 3) with 13 pitches per octave.

In [58]:
intervals = [1, 27/25, 25/21, 9/7, 7/5, 75/49, 5/3, 9/5, 49/25, 15/7, 7/3, 63/25, 25/9]
s = Scale(reference_pitch='C4', reference_frequency=260,
              pitches_per_octave=13, octave_interval=3)
s.tune_all_by_interval(intervals)

In [59]:
s[60]

260

In [60]:
s[73]

780

Harry Partch's 43-Tone Scale:

In [61]:
intervals = [
    1,
    81/80, 33/32, 21/20, 16/15, 12/11, 11/10, 10/9, 9/8, 8/7,
    7/6, 32/27, 6/5, 11/9, 5/4, 14/11, 9/7, 21/16, 4/3, 27/20,
    11/8, 7/5, 10/7, 16/11, 40/27, 3/2, 32/21, 14/9, 11/7, 8/5,
    18/11, 5/3, 27/16, 12/7, 7/4, 16/9, 9/5, 20/11, 11/6, 15/18,
    40/21, 64/33, 160/81
]
s = Scale(reference_pitch='A0', reference_frequency=220,
              pitches_per_octave=43, octave_interval=2)
s.tune_all_by_interval(intervals)

We can read a list of intervals from a scala file. The scala file in the folder were allegedly used by Richard D. James (Aphex Twin) to tune the Korg Monologue.

In [3]:
from adaptivetuning import read_scala

filename = "scala_files/afx004.scl"
intervals = read_scala(filename)

s = Scale()
s.reference_pitch = 0
s.reference_frequency = s[0]
s.tune_all_by_interval(intervals, pitches=range(0,128))